In [5]:
import mlflow
from mlflow.tracking import MlflowClient
import os

In [6]:
mlflow_tracking_path = os.path.abspath(os.path.join(os.getcwd(),os.path.pardir,'mlruns'))
mlflow.set_tracking_uri("file:\\"+mlflow_tracking_path)

In [7]:
EXPERIMENT_NAME = "COPD_classifier_experiments"
MODEL_NAME = "COPD_Classifier"

In [8]:
client = MlflowClient()
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

In [15]:
runs = client.search_runs(exp.experiment_id,order_by=["metrics.accuracy DESC"])
best_run = runs[0]
best_acc = best_run.data.metrics["accuracy"]
best_run

<Run: data=<RunData: metrics={'accuracy': 0.267}, params={'criterion': 'entropy', 'max_depth': '7'}, tags={'mlflow.runName': 'COPD_DecisionTreeClassifier',
 'mlflow.source.name': 'c:\\Users\\negra\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'negra'}>, info=<RunInfo: artifact_uri='file:\\c:\\Users\\negra\\mymlprojects\\loka\\mlruns/123292321147107970/25e48b29760a4292a5ca17ddd9b69082/artifacts', end_time=1762341310242, experiment_id='123292321147107970', lifecycle_stage='active', run_id='25e48b29760a4292a5ca17ddd9b69082', run_name='COPD_DecisionTreeClassifier', start_time=1762341305069, status='FINISHED', user_id='negra'>, inputs=<RunInputs: dataset_inputs=[], model_inputs=[]>, outputs=<RunOutputs: model_outputs=[<LoggedModelOutput: model_id='m-a114da4e063d4eee9196c60e1993f4bb', step=0>]>>

In [16]:
run_id = best_run.info.run_id

In [14]:
model_uri=f"runs:/{run_id}/COPD_Classifier"
registered_model = mlflow.register_model(model_uri,MODEL_NAME)

Successfully registered model 'COPD_Classifier'.
2025/11/05 12:28:47 WARNING mlflow.tracking._model_registry.fluent: Run with id 25e48b29760a4292a5ca17ddd9b69082 has no artifacts at artifact path 'COPD_Classifier', registering model based on models:/m-a114da4e063d4eee9196c60e1993f4bb instead
Created version '1' of model 'COPD_Classifier'.


In [17]:
model_version = registered_model.version

In [ ]:
prod_models = [m for m in client.get_latest_versions(MODEL_NAME, stages=["Production"])]
if prod_models:
    current_acc = float(prod_models[0].tags.get("accuracy", 0))
    if best_acc > current_acc:
        client.transition_model_version_stage(MODEL_NAME, model_version, "Production", archive_existing_versions=True)
        client.set_model_version_tag(MODEL_NAME, model_version, "accuracy", str(best_acc))
        print(f"Promoted v{model_version} to Production (acc={best_acc:.4f})")
    else:
        print("No promotion — current Production is better.")
else:
    client.transition_model_version_stage(MODEL_NAME, model_version, "Production")
    client.set_model_version_tag(MODEL_NAME, model_version, "accuracy", str(best_acc))
    print(f"First Production model: v{model_version}")